<a href="https://colab.research.google.com/github/tomonari-masada/course2023-intro2ml/blob/main/08_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 線形回帰
* 線形モデルを使って、数値を予測する問題を解く。

* scikit-learnの線形回帰を利用
 * https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'

## toy dataで線形回帰の使い方を把握

In [ ]:
fig = plt.figure(figsize=(10,7))
x = [2, 1, -3, -1]
y = [-4, -2, 5, 2]
plt.plot(x, y, '.')
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression() 

In [ ]:
reg.fit(x, y) 

In [ ]:
x = np.array(x)
X = x.reshape(-1,1)
print(X)

In [ ]:
reg.fit(X, y)

* x軸上に細かく点を打って各点での予測値をプロット
 * 観測データに近い位置に直線が引かれることを確認する。

In [ ]:
fig = plt.figure(figsize=(10,7))
xs = np.linspace(-3,2,101)
ys = reg.predict(xs.reshape(-1,1))
plt.plot(x, y, '.')
plt.plot(xs, ys)
plt.show()

## 例題: 住宅価格の予測
### California housing prices
* よく例題として使われるデータセット
 * https://www.kaggle.com/camnugent/california-housing-prices

### データセットの取得

In [ ]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing()

In [ ]:
X = data.data
y = data.target

In [ ]:
data.feature_names

In [ ]:
X[0]

### 訓練データ/検証データ/テストデータに分割

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

### 通常の最小二乗法
* 最小二乗法は・・・
 * モデルの出力$\hat{y}_i$と、観測されている目的変数の値$y_i$との差$\hat{y}_i - y_i$をとり・・・
 * それを二乗$(\hat{y}_i - y_i)^2$し・・・
 * 全てのデータ点にわたってその2乗の和$\sum_{i=1}^N (\hat{y}_i - y_i)^2$をとり・・・
 * この和を最小化することで、線形モデルの係数や切片を求める方法。

In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

In [ ]:
y_valid_predict = reg.predict(X_valid)

In [ ]:
print(y_valid_predict)

In [ ]:
print(y_valid)

* R-squared（決定係数）によって評価する
 * 1.0に近いほど良い。

In [ ]:
print(f'R^2: {reg.score(X_valid, y_valid)}')

* 機械学習の世界では、R^2ではなく、直接、予測値の良し悪しを評価する場合がほとんど
 * 回帰の場合は真の値と予測値との差の二乗の平均のルート(RMSE; root-mean-square error)を評価によく使う。

In [ ]:
print(f'RMSE: {np.sqrt(np.mean((y_valid - y_valid_predict) ** 2))}')

* RMSEは、自分でコードを書かなくても、sklearnに関数が用意されている

In [ ]:
from sklearn.metrics import mean_squared_error
print(f'RMSE: {mean_squared_error(y_valid, y_valid_predict, squared=False)}')

# 線形モデルにおける正則化
* 正則化とは、わざとモデルをシンプルなものにすることで・・・
* モデルの係数や切片が、訓練データだけにピッタリ合いすぎることを防ぐ手法を言う。
 * 訓練データだけにピッタリ合ってしまうと、訓練データ以外のデータで予測がうまくいかなくなる。

### リッジ回帰
* 最小二乗法だが、同時に、係数の２乗の和が大きくならないようにするモデル
 * https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

### Lasso
* 最小二乗法だが、同時に、係数の絶対値の和が大きくならないようにするモデル
 * https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

## テストデータを使った最終的な評価
* 通常の最小二乗法、リッジ回帰、Lassoなどで試行錯誤し・・・
* 検証データでの評価値が最も良かったモデルについて、最後に一回、テストデータで評価する。
* テストデータの評価値を見て、試行錯誤に戻ってはいけないです。
 * 論文などで報告するのは、テストデータでの評価値。